In [1]:
# Interpret Q&As. Chinese word segmentation, parsing, prepare for machine learning

import os
import pandas as pd
import jieba
import jieba.analyse

In [5]:
datafile = sorted([x for x in os.listdir('../data') if \
                   x.startswith('data') and x.endswith('.csv')])[-1] # read in latest data
df = pd.read_csv('../data/' + datafile, encoding='utf-8', index_col=0)
print(datafile, 'read in.')

data-2016-09-24-21.19.39.csv read in.


float

In [21]:
def beforeAfter(s):
    seg_list = jieba.cut(s)
    print('\nBefore:\n' + s)
    print('\nAfter:\n' + '/'.join(seg_list))
    
test = frame.ix[0]['symptom']
beforeAfter(test)


Before:
目前情况：1，不和同龄的孩子玩。主要没有兴趣，有时看见大孩子跑闹也想参与，但不知怎么去做。在大人的提示下可以和同龄孩子打招呼，握手，拥抱。。2，能够和大人快乐的玩耍，有要求，有疑问会向大人提问。吐字清楚。如：妈妈，爷爷做什么去了？但不会和大人进行一些比较社会的交流。比如你今天去海洋馆看见什么了？为什么哭了？但是会回答：有没有看见海龟等问题。偶尔也能比较吃力的描述一段刚刚发生的事，语言组织能力不灵活，句子大部分都是引用大人，或电视里听过的。基本都在场景中。3，精细动作差，拿笔划线无力，只会画圈。拿勺子吃饭僵硬不灵活，不会使用工具，除非特意去教。大运动，机械记忆较好，会骑儿童三轮车。6个月就能指认一些字。会唱儿歌，能够跟着老师跳一些简单舞蹈。自己不会单独跳。喜欢看认知型的动画片，比如天线宝宝，巧虎，米奇妙妙屋。爱反复看同一集。不爱看社会行为型的动画片：如粉红猪小妹，我是悠悠。不爱听故事。可能听不懂。偶尔有一两个听下去了。就反复听那一个。4行为刻板，会在某一段时间，坚持某个行为轨迹，改变会大哭大闹。喜欢风扇，转轮子。5，情感对大人依赖，没有安全感，一岁之前没发现这些，除了睡眠不好，很少哭闹，爱玩乐，有时还会逗陌生人玩，对母亲很亲，经常爱笑，会主动亲母亲，看见母亲下班会高兴跑过来。母乳依赖强，每天晚上要吃母乳6～7次，基本吃两口就睡了。哪怕摔破头，只要给予母乳安慰就不哭了。一周岁时，母亲开始值夜班，强制性断奶，母亲离家一周。大宝哭了近两个星期。情绪反常近2个月，直到开始愿意吃奶瓶才有好转，但是性情没有之前平稳，爱急躁，哭闹，不能等待。2岁半之前，二宝出生前还能够接受。那时经常更换抚养人，但是父母下班都是带着玩的，二宝出生时当时父母离家半月在外地住院待产。带小宝回来一个月后。大宝开始情绪失常，经常哭闹，很难哄好，夜间哭闹，每夜闹好几次。明显焦虑，抑郁。后来母亲给予多关注，让父亲陪他睡后，约半年后逐渐好转。那时依赖妈妈，和小弟弟抢妈妈，妈妈上班会哭闹，要求和妈妈一起去，看见妈妈下班会飞奔扑上去。5个月前妈妈外出两个月后，就不要妈妈了，(因中间又开始夜间哭闹被爸爸多次训斥，就跑去和爷爷睡了)，目前依赖爷爷，让妈妈走。睡觉必须要爷爷陪伴，吃饭，吃饭睡觉不见爷爷，会紧张不安哭闹很久，哪怕爷爷在家，醒来不见爷爷在床上也会哭闹，爷爷喊他自己出来也不行。非得爷爷过去。和父

In [24]:
jieba.analyse.extract_tags(test, withWeight=True)

[('哭闹', 0.24611404264772585),
 ('爷爷', 0.22834251369595016),
 ('妈妈', 0.18379381714654208),
 ('大人', 0.11764099291140187),
 ('母亲', 0.10888028065981306),
 ('母乳', 0.09695623483551402),
 ('依赖', 0.08678298960922119),
 ('看见', 0.07921039954859813),
 ('下班', 0.07805235109420561),
 ('二宝', 0.0744845327283489),
 ('动画片', 0.06262012492398754),
 ('同龄', 0.062488951034890966),
 ('大宝', 0.06211122753426792),
 ('吃饭', 0.06022638408224299),
 ('父母', 0.059701496329906546),
 ('离家', 0.054833714792211836),
 ('经常', 0.05445468631635514),
 ('行为', 0.05034101840728972),
 ('之前', 0.04853762127700935),
 ('孩子', 0.048423927817476634)]

In [25]:
jieba.analyse.textrank(test, withWeight=True)

[('爷爷', 1.0),
 ('妈妈', 0.9021897662991276),
 ('哭闹', 0.8919115792066067),
 ('大人', 0.8556454183936896),
 ('母亲', 0.8112763781262181),
 ('看见', 0.7326841665119904),
 ('吃饭', 0.5970104275626192),
 ('开始', 0.5630963372934767),
 ('行为', 0.5583041672657686),
 ('社会', 0.5133030986530981),
 ('不会', 0.4588479378070064),
 ('依赖', 0.44445791430613973),
 ('下班', 0.4112026106406721),
 ('离家', 0.41040473886223205),
 ('没有', 0.4089467922073848),
 ('能够', 0.40697253672681766),
 ('父母', 0.40538612154431924),
 ('喜欢', 0.3994532787848826),
 ('给予', 0.39298663149871993),
 ('不见', 0.3796648082779753)]